In [3]:
!pip install -U langchain langchain-google-genai google-genai

LLM API and setting environmental variable

In [4]:
from getpass import getpass
import os

# Prompt user for Google AI Studio API key (hidden input)
google_api_key = getpass("Please enter your Google API key: ")

# Set it as an environment variable
os.environ["GOOGLE_API_KEY"] = google_api_key

print("Google API key set successfully.")


Please enter your Google API key: ··········
Google API key set successfully.


In [5]:

from typing import List
from pydantic import BaseModel, Field

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

In [6]:
google_studio = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # or "gemini-1.5-pro"
    temperature=0
)

Building an AI system that can analyze customer reviews summaries and perform complex analysis. for each review, use Google Gemini to do the following:

1. Summarize the review in at most 3 lines.
2. Highlight the positives and negatives.
3. Describe the overall sentiment of the reviews (positives, negatives, neutral)
4. Display 3-5 emotions that are expressed in customer review.
5. If statement in positive thank them via email for the reivew.
6. If statement is negative then write a mail to apologize with appropriate response.  





Reviews

In [7]:
# Sample reviews
reviews = [
    "Absolutely thrilled with this laptop! The performance is lightning fast, the display is crisp, and the battery easily lasts over 10 hours. Delivered a day earlier than expected — couldn’t be happier!",
    "Good build quality and smooth performance for everyday tasks. The keyboard is comfortable to type on. Delivery was on time, but the packaging could have been better.",
    "This laptop exceeded my expectations! Boot time is super quick, the screen colors are vibrant, and it handles multitasking effortlessly. Customer support was polite and helpful when I had a query.",
    "Satisfied overall — the laptop runs well for my work needs, and the design feels premium. However, delivery was delayed by two days, which was a bit frustrating.",
    "The laptop is decent for the price. Runs quietly and stays cool even after long hours. Arrived on time, but I wish the speakers were a bit louder."
]

Define Data Structure

In [8]:
# 🛠 Defined data structure
class ReviewAnalysis(BaseModel):
    Summary: str = Field(description="A brief summary of the review in 3 lines")
    Positives: List[str] = Field(description="A list of positives mentioned by the customer in the review")
    Negatives: List[str] = Field(description="A list of negatives mentioned by the customer in the review")
    Sentiment: str = Field(description="One word showing whether the sentiment is positive, negative, or neutral")
    Emotions: List[str] = Field(description="A list of 3-5 emotions expressed by the customer in the review")
    Email: str = Field(description="Detailed email to the customer based on the sentiment")

# 📋 Create parser
parser = PydanticOutputParser(pydantic_object=ReviewAnalysis)


Prompt Template

In [9]:
# Prompt template
prompt_txt = """
Analyze the customer review given below and generate the response based on the instructions mentioned below in the format instructions.
Also remember to write a detailed email response for the email field based on these conditions:
 - Email should be addressed to "Dear Customer" and signed with "Service Agent".
 - Thank them if the review is good.
 - Apologize if the review is bad.

Format instructions:
{format_instructions}

Review:
{review}
"""



# Use from_template for single-template prompt
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
).partial(format_instructions=parser.get_format_instructions())

In [10]:
# Combine into chain
llmchain = prompt| google_studio | parser


In [11]:
import json

# Assuming reviews is a list and you want to process the first review
input_review = reviews[0]

result = llmchain.invoke({"review": input_review})

# Use model_dump() to get a dict (if result supports it)
structured_output = result.model_dump()

# Print the structured output as formatted JSON
print(json.dumps(structured_output, indent=4))

{
    "Summary": "The customer is extremely satisfied with their new laptop. They highlight the impressive performance, crisp display, and long battery life.  The early delivery also contributed to their positive experience.",
    "Positives": [
        "Lightning fast performance",
        "Crisp display",
        "Over 10 hours of battery life",
        "Delivered a day early"
    ],
    "Negatives": [],
    "Sentiment": "Positive",
    "Emotions": [
        "Thrilled",
        "Happy",
        "Satisfied",
        "Excited"
    ],
    "Email": "Dear Customer,\n\nThank you for your wonderful review! We are absolutely delighted to hear that you are thrilled with your new laptop and that you found the performance, display, and battery life to be exceptional. We also appreciate you letting us know about the early delivery.  Your feedback is invaluable to us, and we're so glad we could exceed your expectations.  If you have any further questions or need anything at all, please don't hesi

In [12]:


for review in reviews:
    result = llmchain.invoke({"review": review})
    output = result.model_dump() if hasattr(result, "model_dump") else json.loads(result)
    print(json.dumps(output, indent=4))

{
    "Summary": "The customer is extremely satisfied with their new laptop. They highlight the impressive performance, crisp display, and long battery life.  The early delivery also contributed to their positive experience.",
    "Positives": [
        "Lightning fast performance",
        "Crisp display",
        "Over 10 hours of battery life",
        "Delivered a day early"
    ],
    "Negatives": [],
    "Sentiment": "Positive",
    "Emotions": [
        "Thrilled",
        "Happy",
        "Satisfied",
        "Excited"
    ],
    "Email": "Dear Customer,\n\nThank you for your wonderful review! We are absolutely delighted to hear that you are thrilled with your new laptop and that you've had such a positive experience. We're especially glad to hear about the excellent performance, crisp display, and long battery life.  We strive to provide exceptional products and service, and your feedback is greatly appreciated.  We're also happy to hear the delivery was ahead of schedule!\n\n

In [17]:
import pandas as pd

# Make pandas display full text without truncating
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Process all reviews into structured dicts
structured_reviews = [llmchain.invoke({"review": review}).model_dump() for review in reviews]

# Create DataFrame
df = pd.DataFrame(structured_reviews)

# Show DataFrame (like screenshot format)
display(df)


,Summary,Positives,Negatives,Sentiment,Emotions,Email
0,"The customer is extremely satisfied with their new laptop. They highlight the impressive performance, crisp display, and long battery life. The early delivery also contributed to their positive experience.","[Lightning fast performance, Crisp display, Over 10 hours of battery life, Delivered a day early]",[],Positive,"[Thrilled, Happy, Satisfied, Excited]","Dear Customer,\n\nThank you for your wonderful review! We are absolutely delighted to hear that you are thrilled with your new laptop and that you found the performance, display, and battery life to be exceptional. We also appreciate you letting us know about the early delivery. Your feedback is invaluable to us, and we're so glad we could exceed your expectations. If you have any further questions or need anything at all, please don't hesitate to contact us.\n\nSincerely,\nService Agent"
1,"The customer praised the build quality, smooth performance, and comfortable keyboard. However, they noted that the packaging could have been better. Delivery was on time.","[Good build quality, Smooth performance for everyday tasks, Comfortable keyboard, On-time delivery]",[Packaging could have been better],Positive,"[Satisfaction, Contentment, Slight frustration]","Dear Customer,\n\nThank you for your positive review! We're thrilled to hear that you're happy with the build quality, performance, and keyboard of your new device. We appreciate your feedback regarding the packaging. We are always striving to improve our packaging to ensure the safe arrival of our products. We will take your comments into consideration as we refine our packaging processes. We're glad to hear the delivery was on time. Thank you again for your feedback; we value your business.\n\nSincerely,\nService Agent"
2,"The customer raves about their new laptop, highlighting its fast boot times, vibrant screen, and excellent multitasking capabilities. They also praise the helpfulness of customer support. Overall, a highly positive experience.","[Fast boot time, Vibrant screen colors, Effortless multitasking, Polite and helpful customer support]",[],Positive,"[Joy, Satisfaction, Contentment]","Dear Customer,\n\nThank you for your wonderful review! We are thrilled to hear that you exceeded your expectations with your new laptop and that you found our customer support team helpful. Your feedback is invaluable to us, and we're delighted you're enjoying the fast boot times, vibrant screen, and smooth multitasking capabilities. We appreciate you choosing our product and look forward to serving you again in the future.\n\nSincerely,\nService Agent"
3,"The customer is mostly satisfied with their new laptop. The laptop's performance and design are praised. However, a two-day delivery delay caused frustration.","[Laptop runs well for work needs, Premium design]",[Two-day delivery delay],Positive,"[Satisfaction, Frustration, Contentment]","Dear Customer,\n\nThank you for your feedback! We're thrilled to hear that you're satisfied with the performance and design of your new laptop. We appreciate you taking the time to share your experience.\n\nWe sincerely apologize for the two-day delay in your delivery. We understand this was frustrating, and we are constantly working to improve our shipping processes to ensure timely delivery for all our customers. \n\nIf you have any further questions or concerns, please don't hesitate to contact us.\n\nSincerely,\nService Agent"
4,"The customer found the laptop decent for its price, highlighting its quiet operation and cool temperature even after extended use. However, they wished the speakers were louder. The delivery was timely.","[Decent for the price, Runs quietly, Stays cool even after long hours, Arrived on time]",[Speakers could be louder],Positive,"[Satisfaction, Contentment, Slight disappointment]","Dear Customer,\n\nThank you for your feedback on your recent laptop purchase! We're thrilled to hear that you found it decent for t